In [1]:
import pandas as pd 
from led_models import modules_control,round_1_led
import struct 
import numpy as np

In [2]:
def double_to_float(data):
    for i in range(len(data)):
        if data[i]>10000000:
            packed=struct.pack('>i',int(data[i]))
            data[i] = struct.unpack('>f', packed)[0]

100 .1.1
75 1 1.2
50 1.4 1.5
25 1.8 2.1
10 1.6 1.8

In [3]:
#2.1 10
#1.1 100

#Y = (110 - X)/100 + 1.1

In [3]:
def data_pretreatment(df_number):

    df = pd.read_csv('dataframe'+str(df_number)+'.csv')
    df['平均温度']=(df['1号室内温度']+df['2号室内温度'])/2
    df['平均湿度']=(df['1号室内湿度']+df['2号室内湿度'])/2
    df['time'] = pd.to_datetime(df['time'])
    standard_lights_power, standard_lights_heat = round_1_led()
    led_heat = []
    for i in range(len(df)):
        if df['1号PPFD'][i]<1:
            led_heat.append(0)
        else:
            led_heat.append(standard_lights_heat/1000)
    df['灯具负荷'] = led_heat #kW
    df['自然负荷'] = (df['户外温度'] - df['平均温度']) * 1.1 * 0.5 * 1.005 * 1.29 #kW #循环风机自然进热#循环风机要一直开所以一直有热量进入
    df['新风负荷'] = (110-df['新风机手动开度'])/100 * (df['户外温度'] - df['平均温度']) * 0.5 * 1.005 * 1.29
    df['总热渗入量'] = df['灯具负荷']+df['自然负荷']+df['新风负荷']
    freq = pd.to_timedelta(np.diff(df.time).min())
    freq = 5 * freq
    grouper = df['time'].diff().gt(freq).cumsum()
    #grouped = [x for _,x in df.groupby(grouper)]
    for _,x in df.groupby(grouper):
        x.to_csv('arranged_data'+str(df_number)+'/sub_df' + str(_) + '.csv')


In [6]:
round_1_led()

(1927, 770.8000000000001)

In [7]:
data_pretreatment(1)

In [8]:
data_pretreatment(2)

In [9]:
data_pretreatment(3)

In [4]:
data_pretreatment(4)